In [2]:
!pip install easyocr

  Using cached shapely-2.0.4-cp310-cp310-macosx_10_9_x86_64.whl.metadata (7.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 4.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.6/270.6 kB 2.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 MB 5.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.7/145.7 kB 2.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 5.9 MB/s eta 0:00:0000:0100:01
Using cached shapely-2.0.4-cp310-cp310-macosx_10_9_x86_64.whl (1.4 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.5/313.5 kB 4.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.5/225.5 kB 3.9 MB/s eta 0:00:00:00:01


In [4]:
### This part scans an image of letter and applies an OCR text recognition in order
### to get the text of the image and transforms it to txt and stores it

import os
from PIL import Image
import easyocr

def process_images(image_directory, output_directory):
    """
    Process all images in the specified directory using OCR and save the extracted text to text files.

    Args:
        image_directory (str): Path to the directory containing the image files.
        output_directory (str): Path to the directory where the output text files will be saved.
    """
    # initialize the OCR reader
    reader = easyocr.Reader(['de'])

    # create the output directory if it doesn't exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # iterate through all files in the directory
    for filename in os.listdir(image_directory):
        # check if file is an image
        if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png') or filename.endswith('.pdf'):
            # make the full path to the image file
            image_path = os.path.join(image_directory, filename)

            # open image for OCR
            try:
                with open(image_path, 'rb') as image_file:
                    image_bytes = image_file.read()
            except Exception as e:
                print(f"Error reading {filename}: {e}")
                continue

            # extract text from image using easyocr
            try:
                result = reader.readtext(image_bytes)
            except Exception as e:
                print(f"Error processing {filename}: {e}")
                continue

            # extracted text
            text = '\n'.join([entry[1] for entry in result])

            # define output path to save extracted text
            output_path = os.path.join(output_directory, f'{os.path.splitext(filename)[0]}.txt')

            # check if output file already exists
            if os.path.exists(output_path):
                print(f"Skipping {filename}. Output file {output_path} already exists.")
                continue

            # save extracted text to a text file
            with open(output_path, 'w', encoding='utf-8') as file:
                file.write(text)

            # print a confirmation message about the text extraction
            print(f'Text from {filename} has been saved in: {output_path}.')


In [6]:
# define path to the directory containing the image files
image_directory = "/Users/terezka/code/katia-si/better-letter-API/raw_data/1000014343.jpg" 

# define the path to the directory where the output text files will be saved
output_directory = '/Users/terezka/code/katia-si/better-letter-API/raw_data/german_ocr_text'

# process images in the specified directory
process_images(image_directory, output_directory)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


NotADirectoryError: [Errno 20] Not a directory: '/Users/terezka/code/katia-si/better-letter-API/raw_data/1000014343.jpg'

In [13]:
### this part cleans the text data from the letters from intendation or hyphenation and so on

import os
import re

def clean_extracted_text(text):
    # Handle hyphenated line breaks
    text = re.sub(r'(\w+)-\n(\w+)', r'\1\2', text)
    # Replace remaining newlines with spaces
    text = text.replace('\n', ' ')
    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def clean_and_save_files(input_directory, output_directory):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Iterate through all files in the input directory
    for filename in os.listdir(input_directory):
        # Check if the file is a text file
        if filename.endswith('.txt'):
            # Construct the full path to the input file
            input_file_path = os.path.join(input_directory, filename)
            # Construct the full path to the output file
            output_file_path = os.path.join(output_directory, filename.replace('.txt', '_cldn.txt'))

            # Check if the output file already exists
            if os.path.exists(output_file_path):
                print(f'Skipping {filename}. Output file {output_file_path} already exists.')
                continue

            # Read the content of the input file
            with open(input_file_path, 'r', encoding='utf-8') as file:
                extracted_text = file.read()

            # Clean the extracted text
            cleaned_text = clean_extracted_text(extracted_text)

            # Write the cleaned text to the output file
            with open(output_file_path, 'w', encoding='utf-8') as file:
                file.write(cleaned_text)

            print(f'Cleaned text has been saved to: {output_file_path}')

# Define the input directory containing the extracted text files
input_directory = '/Users/carstenvolland/code/katia-si/better-letter/raw_data/german_ocr_text'

# Define the output directory where cleaned text files will be saved
output_directory = '/Users/carstenvolland/code/katia-si/better-letter/raw_data/german_ocr_text_cleaned'

# Clean and save text files in the input directory
clean_and_save_files(input_directory, output_directory)


Cleaned text has been saved to: /Users/carstenvolland/code/katia-si/better-letter/raw_data/german_ocr_text_cleaned/1000014343_cldn.txt
Cleaned text has been saved to: /Users/carstenvolland/code/katia-si/better-letter/raw_data/german_ocr_text_cleaned/1000014342_cldn.txt
Cleaned text has been saved to: /Users/carstenvolland/code/katia-si/better-letter/raw_data/german_ocr_text_cleaned/letter_school_hamburg_cldn.txt
Cleaned text has been saved to: /Users/carstenvolland/code/katia-si/better-letter/raw_data/german_ocr_text_cleaned/Ist-oder-soll_cldn.txt
Cleaned text has been saved to: /Users/carstenvolland/code/katia-si/better-letter/raw_data/german_ocr_text_cleaned/letter_arbeitsamt_internet_cldn.txt


translated text has been saved to: /Users/carstenvolland/code/katia-si/better-letter/raw_data/english_translation/letter_arbeitsamt_internet_cldn_en.txt
translated text has been saved to: /Users/carstenvolland/code/katia-si/better-letter/raw_data/english_translation/1000014342_cldn_en.txt
translated text has been saved to: /Users/carstenvolland/code/katia-si/better-letter/raw_data/english_translation/1000014343_cldn_en.txt
translated text has been saved to: /Users/carstenvolland/code/katia-si/better-letter/raw_data/english_translation/Ist-oder-soll_cldn_en.txt
translated text has been saved to: /Users/carstenvolland/code/katia-si/better-letter/raw_data/english_translation/letter_school_hamburg_cldn_en.txt


In [15]:
### summarize german text short

from transformers import BartForConditionalGeneration, BartTokenizer
import os

# load pre-trained model and tokenizer
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# function to generate summary
def generate_summary(text):
    inputs = tokenizer([text], max_length=1024, return_tensors='pt', truncation=True)
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, min_length=30, max_length=200, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# define input and output directories
input_directory = '/Users/carstenvolland/code/katia-si/better-letter/raw_data/english_translation'
output_directory = '/Users/carstenvolland/code/katia-si/better-letter/raw_data/english_summary'

# create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# generate summary for each file in the input directory
for filename in os.listdir(input_directory):
    # check if file is a text file
    if filename.endswith('.txt'):
        # construct the full path to the input file
        input_file_path = os.path.join(input_directory, filename)

        # construct the full path to the output file
        output_file_path = os.path.join(output_directory, f'{os.path.splitext(filename)[0]}_sum.txt')

        # check if the output file already exists
        if os.path.exists(output_file_path):
            print(f'skipping {filename}. output file {output_file_path} already exists.')
            continue

        # read the content of the input file
        with open(input_file_path, 'r', encoding='utf-8') as file:
            text = file.read()

        # generate summary
        summary = generate_summary(text)

        # write summary to the output file
        with open(output_file_path, 'w', encoding='utf-8') as output_file:
            output_file.write(summary)

        print(f'summary has been generated and saved to: {output_file_path}')


summary has been generated and saved to: /Users/carstenvolland/code/katia-si/better-letter/raw_data/english_summary/letter_school_hamburg_cldn_en_sum.txt
summary has been generated and saved to: /Users/carstenvolland/code/katia-si/better-letter/raw_data/english_summary/letter_arbeitsamt_internet_cldn_en_sum.txt
summary has been generated and saved to: /Users/carstenvolland/code/katia-si/better-letter/raw_data/english_summary/1000014343_cldn_en_sum.txt
summary has been generated and saved to: /Users/carstenvolland/code/katia-si/better-letter/raw_data/english_summary/Ist-oder-soll_cldn_en_sum.txt
summary has been generated and saved to: /Users/carstenvolland/code/katia-si/better-letter/raw_data/english_summary/1000014342_cldn_en_sum.txt


In [16]:
### trying a longer summarization of the german text


from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import os

# load pre-trained BART model and tokenizer for German
tokenizer = AutoTokenizer.from_pretrained("Shahm/bart-german")
model = AutoModelForSeq2SeqLM.from_pretrained("Shahm/bart-german")

# function to generate summary with adjusted parameters
def generate_summary_longer(text: str) -> str:
    inputs = tokenizer.encode(text, return_tensors="pt", truncation=True, max_length=1024)
    summary_ids = model.generate(
        inputs,
        max_length=400,  # increase max_length to make the summary longer
        min_length=150,  # adjust min_length accordingly
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# define input and output directories
input_directory = '/Users/carstenvolland/code/katia-si/better-letter/raw_data/german_ocr_text_cleaned/'
output_directory = '/Users/carstenvolland/code/katia-si/better-letter/raw_data/german_summary_long/'

# create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# generate summary for each file in the input directory
for filename in os.listdir(input_directory):
    # check if file is a text file
    if filename.endswith('.txt'):
        # construct the full path to the input file
        input_file_path = os.path.join(input_directory, filename)

        # construct the full path to the output file
        output_file_path = os.path.join(output_directory, f'{os.path.splitext(filename)[0]}_sum.txt')

        # check if the output file already exists
        if os.path.exists(output_file_path):
            print(f'skipping {filename}. output file {output_file_path} already exists.')
            continue

        # read the content of the input file
        with open(input_file_path, 'r', encoding='utf-8') as file:
            text = file.read()

        # generate summary
        summary = generate_summary_longer(text)

        # write summary to the output file
        with open(output_file_path, 'w', encoding='utf-8') as output_file:
            output_file.write(summary)

        print(f'summary has been generated and saved to: {output_file_path}')


summary has been generated and saved to: /Users/carstenvolland/code/katia-si/better-letter/raw_data/german_summary_long/letter_arbeitsamt_internet_cldn_sum.txt
summary has been generated and saved to: /Users/carstenvolland/code/katia-si/better-letter/raw_data/german_summary_long/1000014342_cldn_sum.txt
summary has been generated and saved to: /Users/carstenvolland/code/katia-si/better-letter/raw_data/german_summary_long/1000014343_cldn_sum.txt
summary has been generated and saved to: /Users/carstenvolland/code/katia-si/better-letter/raw_data/german_summary_long/Ist-oder-soll_cldn_sum.txt
summary has been generated and saved to: /Users/carstenvolland/code/katia-si/better-letter/raw_data/german_summary_long/letter_school_hamburg_cldn_sum.txt


In [18]:
### translate german summarized tex to english

from transformers import MarianMTModel, MarianTokenizer
import os

# load the MarianMTModel and tokenizer for translation
tokenizer_translate = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-de-en")
model_translate = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-de-en")

# function to translate German text to English
def translate_to_english(german_text):
    # tokenize the German text
    inputs = tokenizer_translate(german_text, return_tensors="pt", padding=True, truncation=True)
    # generate English translation
    translated = model_translate.generate(**inputs)
    # decode the translated text
    translated_text = tokenizer_translate.batch_decode(translated, skip_special_tokens=True)
    return translated_text[0]

# define input and output directories for German summaries
input_directory_german = '/Users/carstenvolland/code/katia-si/better-letter/raw_data/german_summary_long/'
output_directory_english = '/Users/carstenvolland/code/katia-si/better-letter/raw_data/english_summary_long/'

# create the output directory if it doesn't exist
if not os.path.exists(output_directory_english):
    os.makedirs(output_directory_english)

# translate and save summaries for each file in the input directory
for filename in os.listdir(input_directory_german):
    # check if file is a text file
    if filename.endswith('.txt'):
        # read the German summary
        with open(os.path.join(input_directory_german, filename), 'r', encoding='utf-8') as file:
            german_summary = file.read()

        # translate German summary to English
        english_summary = translate_to_english(german_summary)

        # construct the full path to the output file
        output_file_path_english = os.path.join(output_directory_english, filename)

        # write the translated summary to the output file
        with open(output_file_path_english, 'w', encoding='utf-8') as output_file:
            output_file.write(english_summary)

        print(f'translated summary has been saved to: {output_file_path_english}')


Translated summary has been saved to: /Users/carstenvolland/code/katia-si/better-letter/raw_data/english_summary_long/1000014343_cldn_sum.txt
Translated summary has been saved to: /Users/carstenvolland/code/katia-si/better-letter/raw_data/english_summary_long/1000014342_cldn_sum.txt
Translated summary has been saved to: /Users/carstenvolland/code/katia-si/better-letter/raw_data/english_summary_long/letter_school_hamburg_cldn_sum.txt
Translated summary has been saved to: /Users/carstenvolland/code/katia-si/better-letter/raw_data/english_summary_long/letter_arbeitsamt_internet_cldn_sum.txt
Translated summary has been saved to: /Users/carstenvolland/code/katia-si/better-letter/raw_data/english_summary_long/Ist-oder-soll_cldn_sum.txt


In [ ]:
### playground to test several paramters in order to get the best translation


from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load BART model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Shahm/bart-german")
model = AutoModelForSeq2SeqLM.from_pretrained("Shahm/bart-german")

# Define your original text
original_text = """
Schulisches Mittagessen: Neuer Maximalpreis für das Schuljahr 2023/24 und neue Preisobergrenze für Caterer ab dem 01.08.2023 Sehr geehrte Eltern, mit diesem Schreiben möchten wir Sie darüber informieren; dass auf Basis der allgemeinen Preisentwicklung der von den Eltern zu zahlende Höchstpreis für das Mittagessen zum 1. August 2023 um 20 Cent auf 4,35 Euro angehoben wird. Angesichts der nach wie vor hohen Lebensmittelpreise deckt dieser Beitrag derzeit nicht die tatsächlichen Kosten für ein Mittagessen: Die Caterer können daher ab dem 01.08.2023 bis zu 4,80 Euro Mittagessen abrechnen Die Differenz zwischen dem von den Eltern zu zahlenden Maximalpreis von 4,35 Euro und der neuen Preisobergrenze für die Caterer übernimmt die Schulbehörde und rechnet die Differenz direkt mit den Caterern ab. Das gilt für alle Mittagessen; auch die der vollzahlenden Schülerinnen und Schüler. Damit setzt der Hamburger Senat konsequent die seit 2020 begonnene Linie durch; im Sinne der Familien; Kinder und Jugendlichen ein schulisches Mittagessen zu vertretbaren Preisen zu sichern und den an den Hamburger Schulen tätigen Cateringunternehmen angemessene Preise zu ermöglichen: Insgesamt übernehmen die Freie und Hansestadt Hamburg und der Bund deutlich mehr als 50 Prozent der Kosten aller schulischen Mittagessen, um für alle Schülerinnen und Schüler an Hamburgs Schulen ein gesundes Mittagessen zu gewährleisten: Mehr Informationen hierzu finden Sie bei Bedarf unter Mittaqessen_für_die_Hamburqer_Schulen hamburq de. Mit freundlichen Grüßen Ihr Ganztagsreferat pro
"""

# Generate summary with longer max_length
def generate_summary_longer(text: str) -> str:
    inputs = tokenizer.encode(text, return_tensors="pt", truncation=True, max_length=1024)
    summary_ids = model.generate(
        inputs,
        max_length=400,  # Increase max_length to make the summary longer
        min_length=150,  # Adjust min_length accordingly
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Generate longer summary
summary_longer = generate_summary_longer(original_text)

# Print the longer summary
print(summary_longer)


Die Differenz zwischen dem von den Eltern zu zahlenden Maximalpreis von 4,35 Euro und der neuen Preisobergrenze für die Caterer übernimmt die Schulbehörde. Das gilt für alle Mittagessen; auch die der vollzahlendsten Schülerinnen und Schülern. Damit setzt der Hamburger Senat konsequent die seit 2020 begonnene Linie durch. Mit freundlichen Grüßen Ihr Ganzirksame Ganztagsreferat pro state of the sun: Die Freie und Hansestadt Hamburg.
